<a href="https://colab.research.google.com/github/adrianacupp/NLP_with_Transformers/blob/main/NER_multilingual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#!pip install datasets
from datasets import get_dataset_config_names

In [3]:
xtreme_subsets= get_dataset_config_names("xtreme")
print(f"XTREME has {len(xtreme_subsets)} config")
#wikipedia articles in many languages

XTREME has 183 config


In [6]:
#load only PAN config
panx_subsets = [i for i in xtreme_subsets if i.startswith("PAN")]
panx_subsets[:3]

['PAN-X.af', 'PAN-X.ar', 'PAN-X.bg']

In [8]:
#real case scenario: a swiss corpus in en, fr, de, it
#let's create a deafualt dict tha stores the lang code as key and PAN-X dic as values
from datasets import load_dataset
from collections import defaultdict
from datasets import DatasetDict

In [9]:
langs =['de','fr','it','en']
fracs = [0.629,0.229,0.084,0.059]
#return a dsdic if a key doesn't exists
panx_ch = defaultdict(DatasetDict)

In [ ]:
for lang, frac in zip(langs, fracs):
  #load monolingual corpus
  ds= loa
